#### Digital Signal Processing Courseware: An Introduction (copyright © 2024)
## Authors: J. Christopher Edgar and Gregory A. Miller

Originally written in Mathematica by J. Christopher Edgar. Conversion to Jupyter Notebook by Song Liu.

The authors of this courseware are indebted to Prof. Bruce Carpenter (University of Illinois Urbana-Champaign). Bruce inspired the creation of this courseware, he consulted with the authors as this courseware was being developed, and he provided the original version of the code and text for several sections of this courseware (e.g. the section on complex numbers and the section on normal distributions). 

# <font color=red>DSP.01 Frequency, Amplitude, and Phase</font>

# <font color=red>TUTORIAL</font>

### Setup

In [ ]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image as img
from matplotlib import cm
from mpl_toolkits import mplot3d
from scipy.fft import fft, fftfreq
import matplotlib.patches as patches
import math
import cmath
import pandas as pd
from sympy import Symbol, sin, series
from sympy import roots, solve_poly_system
import scipy.special

import warnings
warnings.filterwarnings('ignore')

# Figure size 
plt.rc("figure", figsize=(8, 6))

#function to create time course figure
#one waveform
def make_plot_1(x1,y1,type="b",linewidth = 1): 
    plt.plot(x1, y1,type)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#two overlaid waveforms with red and blue   
def make_plot_2(x1,y1,type1,x2,y2,type2): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#three overlaid waveforms with red, blue and green   
def make_plot_3(x1,y1,type1,x2,y2,type2,x3,y3,type3): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.plot(x3, y3, type3)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
def make_plot_3d(ax,x,y,z):    
    ax.contour3D(x, y, z, 50, cmap=cm.coolwarm)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
def make_plot_freq_1(x1,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf = fft(x1)
    xf = fftfreq(N, 1 / sample_rate)
    yf = yf[0:Nhalf]
    xf = xf[0:Nhalf]
    plt.plot(xf, np.abs(yf))
    
#two spectrums
def make_plot_freq_2(x1,x2,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf1 = fft(x1)
    yf2 = fft(x2)
    xf = fftfreq(N, 1 / sample_rate)

    yf1 = yf1[0:Nhalf]
    yf2 = yf2[0:Nhalf]
    xf = xf[0:Nhalf]

    plt.plot(xf, np.abs(yf1))
    plt.plot(xf, np.abs(yf2), color = 'r')
    
def make_imshow(x):
    plt.imshow(x,cmap='Greys_r')
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def make_imshow_color(x):
    plt.imshow(x)
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def round_complex(x):
    return complex(np.round(x.real,4),np.round(x.imag,4))


The first few Tutorial sections in this lesson introduce some mathematical facts that we'll use in later lessons to do some of the heavy lifting. If you haven't taken a lot of math classes, some parts of these sections may be difficult to understand at first. Don't worry! We'll revisit these ideas many times in later lessons. If you work through these sections and ask questions when uncertain or confused, then YOU WILL soon understand these important mathematical ideas. Keep your eyes on the prize: big payoffs later for engaging this stuff now.

In the final Tutorial sections, we'll learn how to import data and image files. These later sections are easier, and by the end of this Tutorial you'll be able to import your own data and images and do interesting things to them - and understand what you're doing.

## <font color=red>DSP.01.T1) Identities</font>

### <font color=red>DSP.01.T1.a) Identities - Polynomial Expansions</font>

Sine and cosine functions are close cousins, essentially the same function except for a 90-degree (= $\pi/2$) phase difference. In later lessons, as we examine more advanced topics, we will need to have a good understanding of several identities relating sine/cosine functions and complex exponentials. (We don't really need complex exponentials themselves for work on signal processing, but the tools already developed to work with them are valuable for working with sine/cosine functions. These tools include concepts, algorithms, and software already developed. Might as well save ourselves some work later with a little investment up front.) The similarity between sine/cosine functions and complex exponentials will be examined over the next few lessons. Eventually, once we're familiar with these identities, we'll see that, instead of using sines and cosines, we'll often prefer using complex exponentials.

Here, an expansion is an algebraic way to compute values for the 'Sin' or 'Cos' function. In other words, one can compute a good approximation of those trigonometric relationships with algebra. At some point you might have memorized the polynomial expansions of sine and cosine. If not, here they are:

$\rightarrow$ The expansion of Sin[x] in powers of x (where x is in radians) is

$X-\frac{x^3}{3!}+\frac{x^5}{5!} -...+\frac{(-1)^kx^{2k+1}}{(2k+1)!}+...$

$\rightarrow$ The expansion of Cos[x] in powers of x is

$1-\frac{x^2}{2!}+\frac{x^4}{4!} -...+\frac{(-1)^kx^{2k}}{(2k)!}+...$

Notice that each term follows a simple pattern of: x = current distance along the sine or cosine function (i.e., the phase in radians); raise that x to a power 'k' for the numerator of the next term; and compute k! ('k' factorial) for the denominator.

Also notice that the expansions for sine and cosine are very similar - using odd or even numbers.

These polynomial expansions can be used to plot sine and cosine waves.

How good is the approximation? That's up to you, because it depends on how long that equation is = how many terms you include. The more such terms you include, the more computational time it takes to compute each value, but the more accurate the approximation.

Here is the polynomial expansion of Sin[x] out to the 19th order:

$X-\frac{x^3}{3!}+\frac{x^5}{5!}-\frac{x^7}{7!}+\frac{x^9}{9!}-\frac{x^{11}}{{11}!}+\frac{x^{13}}{{13}!}-\frac{x^{15}}{{15}!}+\frac{x^{17}}{{17}!}-\frac{x^{19}}{{19}!}$

Plot this polynomial. When you look over the Python code, remember that x is in radians. In the code here, we use the variable name "rad" as the x value, and as that value increases we march along the sine function that we're approximating.

In [ ]:
rad = np.arange(0,2*np.pi, 0.001)

timeseries = rad - rad**3/np.math.factorial(3) + rad**5/np.math.factorial(5) - rad**7/np.math.factorial(7)+ \
rad**9/np.math.factorial(9)- rad**11/np.math.factorial(11) + rad**13/np.math.factorial(13)- rad**15/np.math.factorial(15)+ \
rad**17/np.math.factorial(17)-rad**19/np.math.factorial(19)

# Plotting time vs amplitude using plot function 
make_plot_1(rad,timeseries)

plt.show()

Looks like a 1 Hz sine wave. Just to make sure, compare a plot of f[x] = Sin[x] and the polynomial expansion of Sin[x].

In [ ]:
rad = np.arange(0,2*np.pi, 0.001)
freq = 1

timeseries1 = np.sin(freq * rad)
timeseries2 = rad - rad**3/np.math.factorial(3) + rad**5/np.math.factorial(5) - rad**7/np.math.factorial(7)+ \
rad**9/np.math.factorial(9)- rad**11/np.math.factorial(11) + rad**13/np.math.factorial(13)- rad**15/np.math.factorial(15)+ \
rad**17/np.math.factorial(17)-rad**19/np.math.factorial(19)

# Plotting time vs amplitude using plot function 
make_plot_1(rad,timeseries1)
plt.text(np.pi*2,0,'1 sec',fontsize=15)
plt.show()

make_plot_1(rad,timeseries2)
plt.text(np.pi*2,0,'1 sec',fontsize=15)
plt.show()

Pretty conclusive visual evidence that Sin[x] and the polynomial expansion of Sin[x] are identical, within the resolution of this plot. For higher resolution, you could add more terms to the polynomial, but we see that 10 terms already does quite well. If you want a perfect fit, all you have to do is use an infinite number of terms (which of course would take an infinite amount of time to compute). Not doing too many terms usually isn't much of a problem, because each one you add helps less and less.

Use the polynomial expansion of Sin[x] to plot a 3 Hz sine wave. (Notice where we put the "3" in the expansion: we multiply every term by 3, so that the result is a 3 Hz sine wave.) 

In particular, use the 'series' method to quickly derive the polynomial expansion of Sin[3x].

**Note: In the below output, Python simplifies each fraction. So, for example, the second term is $\frac{27x^3}{6}$ which reduces to the shown $\frac{9x^3}{2}$.**

In [ ]:
x = Symbol('x')
series(sin(3*x),x,0,20) 

Plot this polynomial function. 

In [ ]:
rad = np.arange(0,2*np.pi, 0.001)
rad3 = 3 * rad

# Finding amplitude at each time
timeseries = rad3 - rad3**3/np.math.factorial(3) + rad3**5/np.math.factorial(5) - rad3**7/np.math.factorial(7)+ \
rad3**9/np.math.factorial(9)- rad3**11/np.math.factorial(11) + rad3**13/np.math.factorial(13)- rad3**15/np.math.factorial(15)+ \
rad3**17/np.math.factorial(17)-rad3**19/np.math.factorial(19)

make_plot_1(rad,timeseries)

plt.show()

That doesn't look good - not like a sine wave! What went wrong?

Answer: For higher frequencies, more polynomial terms are needed. Try a 35th-order polynomial expansion of Sin[x].

In [ ]:
rad = np.arange(0,2*np.pi, 0.001)
rad3 = 3 * rad

f_range = np.arange(0,18) # 2*(18-1) + 1 = 35
f_range_size = f_range.size
f_range = f_range.reshape((f_range_size,1))

# Finding amplitude at each time
timeseries = (-1)**f_range * rad3**(2*f_range+1) / scipy.special.factorial(2*f_range+1)
timeseries = timeseries.sum(axis = 0)

make_plot_1(rad,timeseries)

plt.show()

Still not right. Try the 51st-order polynomial expansion of Sin[x].

In [ ]:
rad = np.arange(0,2*np.pi, 0.001)
rad3 = 3 * rad

f_range = np.arange(0,26) # 2*(26-1) + 1 = 51
f_range_size = f_range.size
f_range = f_range.reshape((f_range_size,1))

# Finding amplitude at each time
timeseries = (-1)**f_range * rad3**(2*f_range+1) / scipy.special.factorial(2*f_range+1)
timeseries = timeseries.sum(axis = 0)

make_plot_1(rad,timeseries)

plt.show()

Calculating high-order polynomial expansions is a lot of work (you would not want to do that by hand), but it gets the job done.

### <font color=red>DSP.01.T1.b) Identities - the imaginary number ⅈ and complex numbers</font>

In later sections we'll encounter complex numbers. Take a second to recall some important facts about complex numbers.

The imaginary number $i$ (sometime written as upper case $I$ or as J or $j$) enters the story as a solution of
 ${x^2}+1=0$:

In [ ]:
from sympy import *
x = symbols('x')

solve(x**2 + 1, x)

Although you can think of $i$ as the official symbol for $\sqrt{-1}$, fields of science and engineering that work with electrical current generally use $i$ or I for current, so they typically use $j$ or J for $\sqrt{-1}$.

Once $i$=$\sqrt{-1}$ comes in the door, then so do complex numbers. A complex number is any number of the form
a + $i$ b
where a and b are real numbers. Complex numbers come up as solutions of quadratic equations that have no real-number solutions:

In [ ]:
from sympy import *
x = symbols('x')

solve(x**2 + 0.5*x +1, x)

This tells you that the complex numbers -0.25 + 0.968246 $i$ and -0.25 - 0.968246 $i$
solve the equation

${x^2}+0.5x+1=0$ .

### <font color=red>DSP.01.T1.c) Formulas for adding, subtracting, multiplying, and dividing complex numbers</font>

Here are the formulas for the addition, subtraction, multiplication, and division of complex numbers. Remember that something like "a + b ⅈ" is a single complex number (with a real part and an imaginary part), not two numbers.


Addition:

(a + b $i$) + (c + d $i$) = (a + c) + (b + d)$i$

Subtraction:

(a + b$i$) - (c + d $i$) = (a - c) + (b - d)$i$

Multiplication:

(a + b $i$) (c + d$i$) = (a c - b d) + (a d + b c)$i$

Division:

$\frac{a + b ⅈ}{c + d ⅈ} = \frac{a + b ⅈ}{c + d ⅈ} \times \frac{c - d ⅈ}{c - d ⅈ} = \frac {ac+b d}{c^2+d^2}+\frac{(b c-a d)  ⅈ}{c^2+d^2}$

In [ ]:
from sympy import Symbol,symbols
from sympy import *

a,b,c,d = symbols('a,b,c,d',real=True)
simplify(a+I*b+c+I*d)

Using the above rules, calculate the following:
(3 + 4$i$) + (5 - 2$i$)

Answer: Simple

(3 + 4$i$) + (5 - 2$i$) = 3 + 5 + 4$i$ - 2$i$ = 8 + 2$i$  

Calculate the following:
(3 + 4$i$) (5 - 2$i$)

Answer: Simple

(3 + 4$i$) (5 - 2$i$) = 15 + 20$i$ - 6$i$ - 8 $i^2$ = 23 +14$i$
 
(remember, $i^2=-1$)

### <font color=red>DSP.01.T1.d) Identities - Euler Identities</font>

We will see in a moment that the Euler Identities are a help to us in working with sine and cosine functions as a way to work with vectors of signals. The Euler Identities are:

 $e^{i x}$ = cos[x] + $i$sin[x]

and

 $e^{-i x}$ = cos[x] - $i$sin[x]

where $i$ = $\sqrt{-1}$ 

Look at the polynomial expansion of $e^x$ to verify this identity:

The expansion of $e^x$ in powers of x is:

 $e^x = 1+x+\frac{x^2}{2!}+\frac{x^3}{3!}+\frac{x^4}{4!}+\frac{x^5}{5!}+...\frac{x^k}{k!}+... $

A famous mathematician, Euler had the idea of letting x be a complex number ($i$ x). When we substitute ($i$x) for x we get:


 $e^{ix} = 1+ix+\frac{(ix)^2}{2!}+\frac{(ix)^3}{3!}+\frac{(ix)^4}{4!}+\frac{(ix)^5}{5!}+...\frac{(ix)^k}{k!}+... $

Recalling that $i^2=-1$ and that $i^4=1$ , we obtain:

 $e^{ix} = 1+ix-\frac{x^2}{2!}+\frac{(ix)^3}{3!}+\frac{x^4}{4!}+\frac{(ix)^5}{5!}+...\frac{(ix)^k}{k!}+... $

You can see that every other term is an imaginary number (has an $i$ in it). Separating the real and imaginary parts of that expansion and regrouping, we obtain:

 $e^{ix} = (1-\frac{x^2}{2!}+\frac{x^4}{4!}-...) + i(x-\frac{x^3}{3!}+\frac{x^5}{5!}+...)$

If you look closely, you'll see the corresponding sine and cosine expansions that we worked out earlier in this Lesson 1 Tutorial, such that:

$e^{i x}$ = cos[x] + $i$ sin[x]. 

Using the Euler identities, what is the value of $e^{i \pi}$ ?

Answer:

$e^{i \pi} = cos[\pi]$ + $i$ $sin[\pi]$ = -1 + $i$ $\times$ 0 = -1.

### <font color=red>DSP.01.T1.e) Exponential Rules</font>

The rules that govern real exponential functions also govern exponentials with complex terms. To remind you of some basic rules when exponents have a common base (no matter what the base is, but here using $e$ as the base):

 $e^x  e^y = e^{x+y} $

 $\frac{1}{e^x}= e^{-x} $

 $\frac{e^x}{e^y} = e^{x-y}$ 

### <font color=red>DSP.01.T1.f) Euler Identities Continued</font>

Here are the Euler identities:

 $e^{i x}$ = cos[x] + $i$ sin[x] and

 $e^{-i x}$ = cos[x] - $i$ sin[x]$

Use the Euler identities to derive an equation in terms of sin[x]:

Hint: subtract the two identities to solve for sin[x]

 $e^{i x}-e^{-i x} = (cos[x] + i sin[x]) - (cos[x] - i sin[x])$
 
 $e^{i x}-e^{-i x} = i sin[x] + i sin[x]$
 
 $e^{i x}-e^{-i x} = 2i sin[x]$
 
 $\frac{e^{i x}-e^{-i x}}{2 i} = sin[x]$
 

By the same method, $cos[x] = \frac{1}{2} (e^{i x}+e^{-i x})$ . You'll derive this identity in the Literacy Sheet. 

Try using the identity $sin[x] = \frac{e^{i x}-e^{-i x}}{2 i}$ to plot sine waves.

**Note: here and throughout most of the courseware, in the Python syntax we will write the imaginary number as '$j$'.**

In [ ]:
time = np.arange(0,1,0.001) #create an array representing 1 second
freq = 1
timeseries = (np.exp(2*np.pi * 1j * time) - np.exp(-2*np.pi * 1j * time)) / (1j * 2); 

make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)
plt.show()

That is a really nice 1 Hz sine wave. 

Without using the Python syntax, the complex exponential formula looks like this: $sin[x] = \frac{e^{i 1 x}-e^{-i 1 x}}{2 i}$. Notice the '1' values in the numerator - this is what we do to create a 1 Hz sine wave. As usual, the '1' values have no impact on the arithmetic, and we could leave out the '1' values if you do want a 1 Hz signal. But what if you want some other frequency?

In this equation, frequency is set by multiplying the exponential power term by the desired frequency. 

How would you go about creating a 4 Hz sine wave?

Answer: Frequency is set by multiplying the exponential power term by the desired frequency.

Here is the 4 Hz sine wave.

In [ ]:
time = np.arange(0,1,0.001) #create an array representing 1 second
freq = 4
timeseries = (np.exp(2*np.pi * freq * 1j * time) - np.exp(-2*np.pi * freq * 1j * time)) / (1j * 2)

make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)
plt.show()   

That is a really nice 4 Hz sine wave. 

Without using the Python syntax, the complex exponential formula looks like this: $sin[x] = \frac{e^{i 4 x}-e^{-i 4 x}}{2 i}$. Notice the '4' values in the numerator - this is what we do to create a 4 Hz sine wave. Frequency is set by multiplying the exponential power term by the desired frequency.

Here is a 15 Hz sine wave.

In [ ]:
time = np.arange(0,1,0.001) #create an array representing 1 second
freq = 15
timeseries = (np.exp(2*np.pi * freq * 1j * time) - np.exp(-2*np.pi * freq * 1j * time)) / (1j * 2)

make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)
plt.show()

That is a really nice 15 Hz sine wave. 

Without using the Python syntax, the complex exponential formula looks like this: $sin[x] = \frac{e^{i 15 x}-e^{-i 15 x}}{2 i}$. Notice the '15' values in the numerator - this is what we do to create a 15 Hz sine wave. Frequency is set by multiplying the exponential power term by the desired frequency.

That's all there is to it.  

### <font color=red>DSP.01.T1.g) Lots of ways to plot the same function</font>

We've looked at three different ways to plot a sine wave: using the 'sin' function $f[x] = sin[x]$, using the polynomial expansion of $sin[x]$, and using the complex exponential formula $sin[x] = \frac{e^{i x}-e^{-i x}}{2 i}$ .

Which way is best?

Answer: It depends. So far we've made do with f[x] = sin[x]. Later on we'll see that, to take advantage of more sophisticated signal processing techniques, we'll often use the complex exponential form. 

R. W. Hamming in his fabulously good book 'Digital Filters' (1989) notes that, although real functions are familiar, and although complex exponentials have a mysterious aura about them, it is simply necessary to get used to the fact that the complex exponentials are the real functions in a slight disguise.

We saw above that the 'i' or 'I' is paired with the cosine function in the Euler Identities. It's not that the cosine function is truly, secretly about imaginary numbers. But, if we set up our equations just right, we can use complex arithmetic (with the sine value in the real position and the cosine value in the imaginary position) to do some cool stuff with sinusoidal signals very effciently.

## <font color=red>DSP.01.T2) Spatial Data</font>

### <font color=red>DSP.01.T2.a) Identities - Spatial Series</font>

In everything we've done so far, we have talked about time. The timeseries plots we've been looking at show time on the x axis and amplitude on the y axis. But the approach we've developed is more general than that. If we instead plot space (distance, such as millimeters or miles) on the x and y axis, we can plot a spatial pattern instead of a temporal pattern.

Look at this image.

In [ ]:
X = np.array([
    [255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0]
])

make_imshow(X)

plt.show() 

Think of the image above as a 2D surface - a wall or floor. We can think of the pattern on the surface as having a spatial frequency. The horizontal bars create an alternating pattern, cycling between black and white 4 times. In the vertical direction, that is a spatial frequency of 4 Hz, if we loosen up our meaning of Hz from "cycles per second" to "cycles per something" (per second, per meter, per page$...$). Now think about the horizontal direction. In that image, there is no change going from left to right, so the spatial frequency is zero: 0 "cycles" per any unit of distance.

Take a look at this plot.

In [ ]:
X = np.array([
    [0,255,0,255,0,255,0,255,0,255],
    [0,255,0,255,0,255,0,255,0,255],
    [0,255,0,255,0,255,0,255,0,255],
    [0,255,0,255,0,255,0,255,0,255],
    [0,255,0,255,0,255,0,255,0,255],
    [0,255,0,255,0,255,0,255,0,255],
    [0,255,0,255,0,255,0,255,0,255],
    [0,255,0,255,0,255,0,255,0,255],
    [0,255,0,255,0,255,0,255,0,255],
    [0,255,0,255,0,255,0,255,0,255]
])

make_imshow(X)

plt.show() 

Again, think of the image as a 2D surface - a wall or floor. This time, the horizontal lines alternate between black and white 5 times. That reflects a spatial frequency of 5 'Hz' in the horizontal direction = along the x axis. Now it's the y axis that has a (spatial) frequency of 0 Hz.

Take a look at this plot.

In [ ]:
X = np.array([
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
])

make_imshow(X)

plt.show()  

The horizontal bars create an alternating pattern, cycling between black and white 10 times. In the vertical direction, that reflects is a spatial frequency of 10 'Hz'.

That is all there is to it. Look around the room and see if you can identify the spatial frequency of other objects - blinds on the window, patterns in the flooring, tables in a row, lines on page, etc$...$

Spatial frequency is an attribute of any object that is periodic across position in space. Whereas Hz (cycles per second) is the standard unit of measurement for a timeseries, there are different units of measurement for spatial frequency. As an example, in the study of visual perception, spatial frequency is commonly expressed as the number of cycles per degree of visual angle. 

## <font color=red>DSP.01.T3) Adding Sine Waves - another look</font>

### <font color=red>DSP.01.T3.a) Identities - Adding timeseries together: another example</font>

We'll go back to talking about 'timeseries', but keep in mind that just about everything we do in the temporal domain we could do the spatial domain.

In the Basics portion of Lesson 1, we saw that, if you sum odd-numbered sine waves, you can approximate a square wave. What shape do you think you would get by summing even-numbered sine waves?

Answer: Start by summing a few even-numbered sine waves together, scaling each individual timeseries by the frequency of the sine wave.

In [ ]:
rad = np.arange(0,2*np.pi, 0.001)

timeseries = 1/2 * np.sin(2 *rad) + 1/4 * np.sin(4 * rad) + 1/6 * np.sin(6 * rad) + \
1/8 * np.sin(8 * rad) + 1/10 * np.sin(10 * rad) + 1/12 * np.sin(12 * rad)

make_plot_1(rad,timeseries)
plt.show()

Pretty clear pattern - and very different from a sine wave OR a square wave. Go ahead and add a few more even terms together to get a prettier picture.

In [ ]:
f_range = np.arange(2,1000,2)
f_range_size = f_range.size
f_range = f_range.reshape((f_range_size,1))

time = np.arange(0,1,0.001)
timeseries = np.sin(2*np.pi * time * f_range) / f_range
timeseries = timeseries.sum(axis = 0)

make_plot_1(time,timeseries)

plt.show()             

Folks usually call this a sawtooth function.

Here's a secret: we can build things from repeating waveforms that aren't sine waves or cosine waves. Whatever we start with are called the "basis" waves. Sines (and cosines) provide a lot of flexibility, with a lot of tools already worked out. But one could use different shapes, for better or worse, in a given context. So actually this isn't a secret - just a glimpse into the future if you pursue digital signal processing.

## <font color=red>DSP.01.T4) Working with real data</font>

### <font color=red>DSP.01.T4.a) Loading Data Files</font>

Let's learn how to load data files into our workspace and how to display such data. We'll start by loading some brain data!

Load a data set. To do this, run the code below: 

In [ ]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

#find and select file "visual.txt"
file_path = filedialog.askopenfilename()
visual = pd.read_csv(file_path,delimiter = "\t", header=None)
visual = visual.to_numpy().flatten()
visual

There are 250 pieces of information in the imported file. The data were collected starting at -500 ms (a convention is to talk about negative time to mean an interval before some important event, such as the onset of a stimulus; so, "-500 ms" can mean that the timeseries began 500 ms before the stimulus event). A data value was collected once every 4 ms from via some measure of a left hemisphere visual cortex brain response. 

(Before we do anything, let's think a bit more about the data we're going to work with: 250 points at 4 ms per point spans 1000 ms = 1 second. Again, the start time is negative to indicate that the vector of data starts with information that occurred before some key event, which occurred at time 0. So, in this case, we have 250 points, of which 125 were collected in the half a second before the event, and the other 125 were collected in the half second after the event.)

Go ahead and plot the data. In additon to the 'visual' data file, we need to create a variable we will use to plot the x-axis time information. This is created below with the 'timeV' variable.  

In [ ]:
timeV = np.arange(-500,500, 4)
 
plt.plot(timeV, visual)

plt.show()

The y axis shows amplitude values (the strength of brain response to a visual stimulus), and the x axis shows 'time' in ms. The plt.plot function is used to plot 'time' and visual waveform. The data start at -500 ms, there are a total of 250 samples, and a sample is collected once every 4 ms. Therefore, activity was recorded from -500 ms to 496 ms.

It's sometimes useful to add labels to a plot to draw a reader's attention to specific parts of the plot. As an example, we can spruce up the above plot by indicating the latency of the two highest peaks.

First, find the value and position of the two highest peaks. Here is one way to do that. 

In [ ]:
maximum = visual.max()
maximum_position = visual.argmax()
[maximum,maximum_position]

In [ ]:
minimum = visual.min()
minimum_position = visual.argmin()
[minimum,minimum_position]

Use the information above to add labels to the plot. In particular, note that, from the position information, you can derive the correct time. For example, the largest positive value occurs at the 192nd point. If each point equals 4 ms, that is $192 \times 4$ ms = 768 ms. However, before you interpret that, you may want to remove the 500 ms baseline that preceded the stimulus onset, as well as an additional 4 ms for the zero point. (To explain the latter issue: the first data point occurs at 0 ms, and the 2nd point occurs 4 ms into the vector, not $2\times4=8$ ms into the vector.) Thus, the latency of the maximum peak is 768 - 500 - 4 = 264 ms after the onset of the stimulus. Follow the same procedure to find that the latency of the largest negative peak is: $171 \times 4$ ms = 684 ms; 684 - 500 - 4 = 180 ms. 

In [ ]:
plt.plot(timeV, visual)

plt.text(timeV[maximum_position],visual[maximum_position],'max',fontsize=15)
plt.scatter(timeV[maximum_position],visual[maximum_position],linewidth=5)

plt.text(timeV[minimum_position],visual[minimum_position],'min',fontsize=15)
plt.scatter(timeV[minimum_position],visual[minimum_position],linewidth=5)
plt.ylim(-60,40)
plt.show()

Notice that we set the y-axis amplitude values to -60 to 40 so that we can easily view the two peaks.

If you're so inclined, change the thickness and color of the plotted line using the 'color' and 'linewidth' keyword arguments.

In [ ]:
# Plotting time vs amplitude using plot function from pyplot
plt.plot(timeV, visual, color = "red", linewidth=2)
plt.text(timeV[maximum_position],visual[maximum_position],'max',fontsize=15)
plt.scatter(timeV[maximum_position],visual[maximum_position],linewidth=5)
plt.text(timeV[minimum_position],visual[minimum_position],'min',fontsize=15)
plt.scatter(timeV[minimum_position],visual[minimum_position],linewidth=5)
plt.margins(x=0, y=0)
plt.axhline(y=0, color='k')
plt.ylim(-60,40)
plt.show()

Add a line to show when the visual stimulus was presented.

In [ ]:
make_plot_1(timeV,visual, "r", linewidth=2)
plt.axvline(x=0, color='k')

plt.text(timeV[maximum_position],visual[maximum_position],'max',fontsize=15)
plt.scatter(timeV[maximum_position],visual[maximum_position],linewidth=5)

plt.text(timeV[minimum_position],visual[minimum_position],'min',fontsize=15)
plt.scatter(timeV[minimum_position],visual[minimum_position],linewidth=5)
plt.ylim(-60,40)
plt.tick_params(labelbottom = True, bottom = True)
plt.show()

The marker at zero indicates that there is little brain activity prior to presenting a visual stimulus, and then significant sinusodial activity following the visual stimuli. The post-stimulus activity doesn't look like a pure sine wave. Probably reflects the summation of activity at multiple frequencies - you saw in the Lesson 1 Basics how that can be done - how we (or the brain) can sum multiple sine waves to produce a single signal.

Finally, to complete the plot, let's add labels (with unit information) to the x axis and y axis.

In [ ]:
make_plot_1(timeV,visual, "r", linewidth=2)
plt.axvline(x=0, color='k')

plt.text(timeV[maximum_position],visual[maximum_position],'max',fontsize=15)
plt.scatter(timeV[maximum_position],visual[maximum_position],linewidth=5)

plt.text(timeV[minimum_position],visual[minimum_position],'min',fontsize=15)
plt.scatter(timeV[minimum_position],visual[minimum_position],linewidth=5)
plt.ylim(-60,40)
plt.tick_params(labelbottom = True, bottom = True)
plt.xlabel("Time (ms)",fontsize=15), plt.ylabel("Amplitude (uV)",fontsize=15)
plt.show()

Not too bad a plot for a first try.

## <font color=red>DSP.01.T5) Working with image files</font>

### <font color=red>DSP.01.T5.a) Loading and Displaying Image Files</font>

In later lessons we will examine spatial images in more detail. Let's take a second to learn one way to read in and examine a spatial image.

Let's load in a black-and-white image. To do this, run the code below: 

In [ ]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

#find and select file "statuesgreysmall.jpg"
file_path = filedialog.askopenfilename()
image = img.imread(file_path)
make_imshow(image)
plt.show()  

This picture was taken by David Edgar in Nassau, Bahamas. The statues are made from the trunks of trees.

The keyword argument setup 'cmap='Greys_r' tells us that this is a grayscale image. This grayscale image is stored in what is called JPEG format. (The filename for a JPEG file often ends in .jpeg or .jpg.)

The image is really just a 2D matrix of numbers. Use the command below to obtain the numerical values used to represent this image (be patient, it may take a few seconds to generate the plot). 

In [ ]:
image.shape     

In [ ]:
image_flattern = np.sort(image.flatten())
plt.plot(image_flattern,'blue')
plt.show() 

The 2D matrix 'image' contains the values used to represent the image. The Numpy method 'shape' indicates that the image is composed of a matrix containing 250*167 = 41,750 values. This matrix is 250 columns by 167 rows (notice that the width of the image (# columns) is more than the height (# rows)). The graph above shows the range of values contained in the image. As indicated on the y axis, the values range from 0 to 255. A '0' value represents pure black, a '255' value represents pure white, and the values in-between represent shades of gray. The grayscale image shown above shows a full range of grayscale values. 